In [1]:
import pandas as pd
import os
import pickle

import networkx as nx

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from src.dataset.dataset_info import datasets
from src.graph.graph_measures import calculate_graph_measures
from src.graph.graph_construction.window_graph import create_weightless_window_graph
from src.graph.graph_construction.session_graph import define_sessions, create_weightless_session_graph

In [2]:
flow_graph = True

session_graph = False

window_graph = False
window_size= 2000

multi_graph=False
line_graph = False
sort_timestamp = False

In [3]:
# name = "cic_ids_2017_5_percent"
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017"
name = "cic_bot_iot"
# name = "nf_bot_iot"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"

dataset = datasets[name]

In [4]:
df = pd.read_parquet(dataset.path)

In [5]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
index,,,,,,,,,,,,,,,,,,,,,
0,192.168.100.150-192.168.100.3-54114-44581-6,192.168.100.150,54114.0,192.168.100.3,44581.0,6.0,21/05/2018 08:34:13 PM,64465.0,0.0,2.0,...,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance,3
1,192.168.100.147-192.168.100.7-1649-80-6,192.168.100.147,1649.0,192.168.100.7,80.0,6.0,04/06/2018 05:38:04 PM,22646986.0,3.0,1.0,...,0.000000e+00,3281987.0,3281987.0,9682499.5,5.878778e+06,13839423.0,5525576.0,1,DDoS,1
2,192.168.100.148-192.168.100.3-59032-80-6,192.168.100.148,59032.0,192.168.100.3,80.0,6.0,04/06/2018 01:41:34 PM,153463.0,1.0,2.0,...,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance,3
3,192.168.100.149-192.168.100.7-2820-80-17,192.168.100.149,2820.0,192.168.100.7,80.0,17.0,04/06/2018 02:29:28 PM,24915470.0,4.0,1.0,...,0.000000e+00,4777031.0,4777031.0,7784938.5,2.586275e+06,9613711.0,5956166.0,1,DDoS,1
4,192.168.100.150-192.168.100.5-5146-80-17,192.168.100.150,5146.0,192.168.100.5,80.0,17.0,04/06/2018 06:17:12 PM,46437692.0,8.0,1.0,...,7.235516e+06,14714089.0,4481524.0,9080693.0,5.048133e+06,14796053.0,5230723.0,1,DDoS,1


In [6]:
cols_to_norm = list(set(list(df.columns ))  - set(list([dataset.label_col]))  - set(list([dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))
cols_to_norm

['Protocol',
 'Bwd IAT Min',
 'Subflow Fwd Byts',
 'Active Min',
 'Idle Min',
 'Active Std',
 'PSH Flag Cnt',
 'Fwd IAT Tot',
 'Bwd Seg Size Avg',
 'ECE Flag Cnt',
 'RST Flag Cnt',
 'Idle Std',
 'Pkt Len Var',
 'Bwd IAT Max',
 'Pkt Len Min',
 'Subflow Bwd Byts',
 'Fwd IAT Min',
 'Bwd Pkts/s',
 'Fwd Act Data Pkts',
 'Fwd Seg Size Avg',
 'URG Flag Cnt',
 'Subflow Bwd Pkts',
 'FIN Flag Cnt',
 'Down/Up Ratio',
 'SYN Flag Cnt',
 'ACK Flag Cnt',
 'Flow IAT Min',
 'Pkt Len Std',
 'Fwd Pkt Len Std',
 'Flow Byts/s',
 'Fwd Pkts/s',
 'Init Bwd Win Byts']

In [7]:
df[dataset.label_col].value_counts()

Label
1    13338328
0       89190
Name: count, dtype: int64

In [8]:
if sort_timestamp:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [9]:
if window_graph and line_graph:
    
    df.reset_index(drop=True, inplace=True)
    df.reset_index(drop=False, inplace=True)
    
    scaler = StandardScaler()

    df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])
    df['h'] = df[ cols_to_norm ].values.tolist()
    
    folder_path="datasets/" + name + "/line_graph_unsorted"
    if sort_timestamp:
        folder_path="datasets/" + name + "/line_graph_sorted"
        
    create_weightless_window_graph(
        df=df,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=True,
        folder_path=folder_path,
        # folder_path=None,
        # test_percentage = 20,
        edge_attr= ['h',dataset.label_col, "index"],
        file_type="pkl")    

In [10]:
if session_graph:
    folder_path="datasets/" + name + "/session_graphs"
        
    df2 = define_sessions(df,
                          src_ip_col=dataset.src_ip_col,
                          src_port_col=dataset.src_port_col,
                          dst_ip_col=dataset.dst_ip_col,
                          dst_port_col=dataset.dst_port_col,
                        #   protocol_col=dataset.
    )
    
    create_weightless_session_graph(df,
                                    src_ip_col = dataset.src_ip_col,
                                    dst_ip_col = dataset.dst_ip_col,
                                    multi_graph=multi_graph,
                                    line_graph=line_graph,
                                    folder_path=folder_path)

In [11]:
if not window_graph:
	y = df[dataset.label_col]
 
	if sort_timestamp:
		X_train, X_test, y_train, y_test = train_test_split(
			df, y, test_size=0.3)
	else:
		X_train, X_test, y_train, y_test = train_test_split(
			df, y, test_size=0.3, random_state=13, stratify=y)

	del df

	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_train.shape: {X_test.shape}")

	scaler = StandardScaler()

	X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
	X_train['h'] = X_train[ cols_to_norm ].values.tolist()

	X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
	X_test['h'] = X_test[ cols_to_norm ].values.tolist()

==>> X_train.shape: (9399262, 85)
==>> X_train.shape: (4028256, 85)


In [12]:
if flow_graph:
    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col], create_using=nx.MultiDiGraph())
    # get netowrk properties
    # graph_measures = calculate_graph_measures(G, "datasets/" + name + "/training_graph_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open("datasets/" + name + "/training_graph.pkl", "wb") as f:
        pickle.dump(G, f)

In [13]:
if flow_graph:
    G_test = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col],create_using=nx.MultiDiGraph())
    
    # graph_measures = calculate_graph_measures(G_test, "datasets/" + name + "/testing_graph_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open("datasets/" + name + "/testing_graph.pkl", "wb") as f:
        pickle.dump(G_test, f)

In [14]:
# if window_graph and line_graph:

#     create_weightless_window_graph(
#         df=X_test,
#         src_ip_col=dataset.src_ip_col,
#         dst_ip_col=dataset.dst_ip_col,
#         window_size=window_size,
#         line_graph=True,
#         folder_path="datasets/" + name + "/line_graph/testing",
#         file_type="pkl")

In [15]:
# nx.write_gexf(G_test, "datasets/cic_ton_iot/testing_graph.gexf")